In [6]:
import torch
import torch.nn as nn
import numpy as np


- softmax with numpy

In [17]:
import torch
import torch.nn as nn
import numpy as np

def softmax(x):
    return np.exp(x) / np.sum(np.exp(x), axis=0)
# 数组 / 标量，用到了向量化操作，可以不用python的循环

x = np.array([2.0, 1.0, 0.1])
outputs_x = softmax(x)
print(f'softmax with numpy:{outputs_x}')

softmax with numpy:[0.65900114 0.24243297 0.09856589]


- softmax with pytorch

In [23]:
import torch
import torch.nn as nn
import numpy as np

y = torch.tensor([2.0, 1.0, 0.1], dtype=torch.float32)
outputs_y_1 = torch.softmax(y, dim=0)    # 要指定轴
outputs_y_2 = y.softmax(dim=0)           # 另一种方法
print(f'softmax_pytorch: \n{outputs_y_1} \n{outputs_y_2}')


softmax_pytorch: 
tensor([0.6590, 0.2424, 0.0986]) 
tensor([0.6590, 0.2424, 0.0986])


- cross-entropy
  数值越低越好，独热编码和概率的乘积

- cross entropy with numpy

In [3]:
import torch
import torch.nn as nn
import numpy as np

def cross_entropy(actual, predicted):
    loss = -np.sum(actual * np.log(predicted))
    return loss # / float(predicted.shape[0])

Y = np.array([1, 0, 0])

Y_pred_good = np.array([0.7, 0.2, 0.1])
Y_pred_bad = np.array([0.1, 0.3, 0.6])
l1 = cross_entropy(Y, Y_pred_good)
l2 = cross_entropy(Y, Y_pred_bad)
print(f'Loss_good with numpy: {l1:.4f}')
print(f'Loss_bad with numpy: {l2:.4f}')



Loss_good with numpy: 0.3567
Loss_bad with numpy: 2.3026


- cross entropy with pytorch
1. nn.crossentropy 同时实现了nn.LogSoftmax + nn.NLLLoss, 所以不要再加softmax层
2. inputs（预测的值）是模型的原始得分（raw scores），这个类会先对inputs应用LogSoftmax类
3. 计算上一步得到的log值与target的负对数似然损失，这里targets是类别的索引而不是独热编码，但索引和独热的效果是一样的，提高了计算效率

In [4]:
import torch
import torch.nn as nn
import numpy as np

loss = nn.CrossEntropyLoss()



